In [17]:
import tweepy
import csv
import pandas as pd
import numpy as np
import networkx as nx

In [7]:
####input your credentials here
consumer_key = '95cMtk1vJvEEW2rlMR0kIU9lE'
consumer_secret = 'pMQFi7LBdcudKDNZOokUJGS8mDxQanUv8spxBDdTLiwSZBuUOM'
access_token = '1036313393114767360-BZ8Qpi02ghRvehhcITEIyl7SmGWmU6'
access_token_secret = 'C7VAqGDhTdB424iBtEwF1CJI9YPTcvNvLjFmaCXENNv3G'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
#####United Airlines
# Open/Create a file to append data
csvFile = open('ua.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

for status in tweepy.Cursor(api.search,q="#blacklivesmatter",count=1,tweet_mode = 'extended',
                           lang="en",
                           since="2017-04-03").items():
    print("Tweet Startd")
    if hasattr(status, "retweeted_status"):  # Check if Retweet
        try:
            print(status.retweeted_status.extended_tweet["full_text"])
        except AttributeError:
            print(status.retweeted_status.full_text)
    else:
        try:
            print(status.extended_tweet["full_text"])
        except AttributeError:
            print(status.full_text)
    print(status.parse())
    print("Tweet Ended")
    #csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

Tweet Startd
BLACK LESBIANS AND BLACK TRANS WOMEN, THIS IS FOR YOU.

leave your c*shapps, v*nmos, p*ypals, gofundmes, ko-fis, patreons down below.

#transcrowdfund #blacklivesmatter
#payblackwomen
#blacktranscrowdfund


TypeError: parse() missing 2 required positional arguments: 'api' and 'json'

In [4]:
class Tweet():
    
    def __init__(self, status_response):
        self.id = status_response.id
        self._get_text(status_response)
        self.user_info = status_response.user
        self.entitites = status_response.entities
        
    def _get_text(self, status):
        if hasattr(status, "retweeted_status"):  # Check if Retweet
            try:
                self.text = status.retweeted_status.extended_tweet["full_text"]
            except AttributeError:
                self.text = status.retweeted_status.full_text
        else:
            try:
                self.text = status.extended_tweet["full_text"]
            except AttributeError:
                self.text = status.full_text
    
    #code for generating named entities of each tweet
    def _get_named_entities(self):
        pass
    #code for generating event phrases
    def _get_event_phrases(self):
        pass
    
    def get_graph_entities(self)
        return self._get_named_entities() + self._get_event_phrases()
    
    def __hash__(self):
        return self.id
    def __eq__(self, other):
        return self.id == other.id
    
class TweetRetriever():
    
    def __init__(self):
        consumer_key = '95cMtk1vJvEEW2rlMR0kIU9lE'
        consumer_secret = 'pMQFi7LBdcudKDNZOokUJGS8mDxQanUv8spxBDdTLiwSZBuUOM'
        access_token = '1036313393114767360-BZ8Qpi02ghRvehhcITEIyl7SmGWmU6'
        access_token_secret = 'C7VAqGDhTdB424iBtEwF1CJI9YPTcvNvLjFmaCXENNv3G'
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        self.api = tweepy.API(auth,wait_on_rate_limit=True)
    
    def getTweets(self, hashtag, count = 10):
        tweets = []
        for status in tweepy.Cursor(self.api.search, q = hashtag, count = count, 
                                    tweet_mode = 'extended', lang = 'en').items():
            tweets.append(Tweet(status))
        return tweets

    

In [ ]:
class GraphNode():
    
    def __init__(self, name):
        self.name = name
        self.tweeets = set()
        self.value = 0
        
    def add_tweet(self, tweet):
        self.tweeets.add(tweet)
    
    def common_tweets(self, other):
        return len(self.tweeets.intersection(other.tweets))
    
class TweetGraph():
    
    def __init__(self, topic):
        self.topic = topic
        self.nodes = []
        self.edge_map = {}
        
    def add_node(self, node):
        self.nodes.append(node)
    
    def add_edge(self, node1, node2):
        assert node1.name in self.node_map
        assert node2.name in self.node_map
        weight = node1.common_tweets(node2)
        self.edge_map.setdefault(node1.name, {}).setdefault(node2.name, weight)
        self.edge_map.setdefault(node2.name, {}).setdefault(node1.name, weight)
    
    def compute_all_edges(self):
        for node1 in self.nodes:
            for node2 in self.nodes:
                self.add_edge(node1, node2)
    
    def _get_pagerank_matrix(self):
        x = [[0 for _ in len(self.nodes)] for _ in len(self.nodes)]
        for i, node1 in enumerate(self.nodes):
            wsum = 0
            for node2 in self.nodes:
                wsum += self.edge_map.get(node1.name, {}).get(node2.name, 0)
            for j, node2 in enumerate(self.nodes):
                x[i][j] = self.edge_map.get(node1.name, {}).get(node2.name, 0)/wsum
        return np.array(x)
    
    def set_textrank_values(self, d = 0.85):
        rank_graph = nx.from_numpy_array(self._get_pagerank_matrix())
        node_scores = nx.pagerank(rank_graph, aplpha = d)
        for n, s in enumerate(node_scores):
            self.nodes[n].value = s
        

In [5]:
ret = TweetRetriever()

In [13]:
tweets = ret.getTweets('#worlds2019', 2)

In [16]:
tweets[1].text

'@TeamGriffinLoL vs @G2esports tiebreaker highlights\U0001f973:\n\n3 min GRF FB\n18 min GRF 3 for 0 mid lane\n21 min GRF 4 for 2 blue side jungle\n27 min GRF ACE in base to win\n\n#lckwin #Worlds2019'